In [1]:
import pandas as pd
from scipy.stats import pearsonr
import os
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateLocator, DateFormatter
from datetime import datetime
import seaborn as sns
import numpy as np
import matplotlib.patches as mpatches
folder = '/Users/yiwengeng/Documents/PregnantPrj/Data/PTL_Data/'
dirlist = ["PTL01","PTL02","PTL03","PTL05","PTL06","PTL07","PTL09","PTL10","PTL11","PTL12","PTL13","PTL14","PTL15","PTL16","PTL17"]
# dirlist = ["PTL02","PTL03"]
act_names=["activity_score", "score_meet_daily_targets","score_move_every_hour","score_stay_active","steps"]
hr_names=["bpm"]
spo2_names=["spo2"]
hrhrv_names=["5min_hr","5min_hrv"]
ready_names=["readiness_score", "score_previous_night","score_sleep_balance", 
             "score_previous_day","score_activity_balance","score_resting_hr",
             "score_hrv_balance","score_recovery_index","score_temperature"]
sleep_names=["sleep_score", "duration","efficiency","onset_latency","rmssd",
             "score_deep","score_efficiency","score_rem"]

df_act = {}
df_sleep = {}
df_ready = {}
df_hr = {}
df_hrhrv = {}
df_spo2 = {}

marker = "cEmotAbuse"
healthy = [1,2,3,8,9,10,12,13,14,15,18]
unhealthy = [4,5,6,7,11,16,17]

for dir in dirlist:
    ID = int(dir[-2:])
    os.chdir(folder+dir)
    files = sorted(os.listdir())
    # print("ID",ID)
    # activity
    df_act[ID] = pd.read_csv(files[0], parse_dates = [0])
    df_act[ID].rename(columns={'score':'activity_score'},inplace=True)
    df_act[ID]['ID'] =  ID
    df_act[ID][marker] = 0 if ID in healthy else 1 if ID in unhealthy else None
        
    # df_act[ID]['Trama'] = 0 if ID in [1,2,3,9,10,11,12,13,14,15,17,18] else 1 if ID in [5,6,7,16] else None
    # print('act',len(df_act[ID]))

    # sleep
    df_sleep[ID] = pd.read_csv(files[7], parse_dates = [0])
    df_sleep[ID].rename(columns={'score':'sleep_score'},inplace=True)
    df_sleep[ID]['ID'] =  ID
    # df_sleep[ID]['Trama'] = 0 if ID in [1,2,3,9,10,11,12,13,14,15,17,18] else 1 if ID in [5,6,7,16] else None
    df_sleep[ID][marker] = 0 if ID in healthy else 1 if ID in unhealthy else None
    
    # print('sleep',len(df_sleep[ID]))
    # readiness
    df_ready[ID] = pd.read_csv(files[5], parse_dates = [0])
    df_ready[ID].rename(columns={'score':'readiness_score'},inplace=True)
    df_ready[ID]['ID'] =  ID
    # df_ready[ID]['Trama'] = 0 if ID in [1,2,3,9,10,11,12,13,14,15,17] else 1 if ID in [5,6,7,16] else None
    df_ready[ID][marker] = 0 if ID in healthy else 1 if ID in unhealthy else None
    # print('ready',len(df_ready[ID]))

    # daily_hr
    df_hr[ID] = pd.read_csv(files[1], parse_dates = [0])
    df_hr[ID]['ID'] =  ID
    # df_hr[ID]['Trama'] = 0 if ID in [1,2,3,9,10,11,12,13,14,15,17] else 1 if ID in [5,6,7,16] else None    
    df_hr[ID][marker] = 0 if ID in healthy else 1 if ID in unhealthy else None    
        
    # print('hr',len(df_hr[ID]))

    # hr_hrv
    df_hrhrv[ID] = pd.read_csv(files[3], parse_dates = [0])
    df_hrhrv[ID].rename(columns={'5-min hr':'5min_hr','5-min hrv':'5min_hrv'},inplace=True)
    df_hrhrv[ID]['ID'] =  ID
    # df_hrhrv[ID]['Trama'] = 0 if ID in [1,2,3,9,10,11,12,13,14,15,17] else 1 if ID in [5,6,7,16] else None   
    df_hrhrv[ID][marker] = 0 if ID in healthy else 1 if ID in unhealthy else None   
    df_hrhrv[ID] = df_hrhrv[ID].loc[df_hrhrv[ID]['5min_hr'] != 0]
    df_hrhrv[ID]['timestamp'] = pd.to_datetime(df_hrhrv[ID]['timestamp'], utc=True)
    df_hrhrv[ID]['days_from_start'] = (df_hrhrv[ID]['timestamp'] - df_hrhrv[ID]['timestamp'].min()).dt.days
    df_hrhrv[ID]['Date'] = pd.to_datetime(df_hrhrv[ID]['timestamp'].dt.date, utc=True)

    # print("hrhrv",len(df_hrhrv[ID]))

    # spo2
    df_spo2[ID] = pd.read_csv(files[2], parse_dates = [0])
    df_spo2[ID].rename(columns={'average spo2':'spo2'},inplace=True)
    df_spo2[ID]['ID'] =  ID
    df_spo2[ID][marker] = 0 if ID in healthy else 1 if ID in unhealthy else None    

    # df_spo2[ID]['Trama'] = 0 if ID in [1,2,3,9,10,11,12,13,14,15,17] else 1 if ID in [5,6,7,16] else None    
    # print('spo2',len(df_spo2[ID]))

    # print("*"*20)

### RMSSD

In [2]:
df_modify_HRV = {}
for dir in dirlist:
    ID = int(dir[-2:])
    # mean_hrv_per_day = df_hrhrv[ID].groupby('Date')['5min_hrv'].mean().reset_index()
    # mean_hrv_per_day['Date'] = pd.to_datetime(mean_hrv_per_day['Date']).dt.date
    # mean_hrv_per_day.rename(columns={'5min_hrv':'Avg_dailyHRV'},inplace=True)
        
    temp_act = pd.DataFrame(data ={'Date':pd.to_datetime(df_act[ID]['summary_date']).dt.date, 'ID': df_act[ID]['ID'],'Trauma': df_act[ID]['Trama'],'Act_Score':df_act[ID]['activity_score']})
    temp_sleep = pd.DataFrame(data ={'Date':pd.to_datetime(df_sleep[ID]['summary_date']).dt.date, 'Sleep_Score':df_sleep[ID]['sleep_score'], 'rmssd':df_sleep[ID]['rmssd']})
    df_modify_HRV[ID] = pd.merge(left = temp_act, right = temp_sleep, on='Date', how='left')
    df_modify_HRV[ID] = df_modify_HRV[ID].loc[df_modify_HRV[ID]['Sleep_Score'].notnull()]

### 5min-HRV

In [3]:
df_modify_HRV = {}
for dir in dirlist:
    ID = int(dir[-2:])
    mean_hrv_per_day = df_hrhrv[ID].groupby('Date')['5min_hrv'].mean().reset_index()
    mean_hrv_per_day['Date'] = pd.to_datetime(mean_hrv_per_day['Date']).dt.date
    mean_hrv_per_day.rename(columns={'5min_hrv':'Avg_dailyHRV'},inplace=True)
        
    temp_act = pd.DataFrame(data ={'Date':pd.to_datetime(df_act[ID]['summary_date']).dt.date, 'ID': df_act[ID]['ID'],marker: df_act[ID][marker],'Act_Score':df_act[ID]['activity_score']})
    temp_sleep = pd.DataFrame(data ={'Date':pd.to_datetime(df_sleep[ID]['summary_date']).dt.date, 'Sleep_Score':df_sleep[ID]['sleep_score']})
    df_modify_HRV[ID] = pd.merge(left = temp_act, right = mean_hrv_per_day, on='Date', how='left')
    df_modify_HRV[ID] = pd.merge(left = df_modify_HRV[ID], right = temp_sleep, on='Date', how='left')
    df_modify_HRV[ID] = df_modify_HRV[ID].loc[df_modify_HRV[ID]['Sleep_Score'].notnull()]

In [4]:
df_modify_HRV_all = pd.concat([df_modify_HRV[int(dir[-2:])] for dir in dirlist], axis=0,ignore_index=True)
print(df_modify_HRV_all.head())


         Date  ID  cEmotAbuse  Act_Score  Avg_dailyHRV  Sleep_Score
0  2023-06-10   1           0         75     47.365854         95.0
1  2023-06-11   1           0         78     80.518519         93.0
2  2023-06-13   1           0         83     40.669492         95.0
3  2023-06-17   1           0         83     49.626016         97.0
4  2023-06-18   1           0         87     12.822222         58.0


In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from scipy.spatial.distance import cdist

logistic_model = LogisticRegression().fit(df_modify_HRV_all[['Sleep_Score', 'Act_Score']], df_modify_HRV_all[marker])
df_modify_HRV_all['Propensity_Score'] = logistic_model.predict_proba(df_modify_HRV_all[['Sleep_Score', 'Act_Score']])[:, 1]

# Quick validation: check the AUC-ROC for the propensity score model
roc_auc = roc_auc_score(df_modify_HRV_all[marker], df_modify_HRV_all['Propensity_Score'])



0.5694534419857478

In [13]:
logistic_model.get_params(deep=True)


{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [7]:
PTL_all = df_modify_HRV_all.groupby(marker)
wo_trauma = PTL_all.get_group(0)
w_trauma = PTL_all.get_group(1)
distances = cdist(w_trauma['Propensity_Score'].values.reshape(-1, 1), wo_trauma['Propensity_Score'].values.reshape(-1, 1), 'euclidean')
# For each treated individual, find the index of the control individual with the smallest distance
matched_indices = np.argmin(distances, axis=1)

# Extract matched control individuals
matched_wo_trauma = wo_trauma.iloc[matched_indices]

# Reset indices for concatenation
w_trauma = w_trauma.reset_index(drop=True)
matched_wo_trauma = matched_wo_trauma.reset_index(drop=True)

# Combine the matched pairs into a single DataFrame
matched_data = pd.concat([w_trauma, matched_wo_trauma], axis=1, keys=['w_trauma', 'wo_trauma'])

# Check the first few rows of matched pairs
matched_data.head()
# os.chdir('/Users/yiwengeng/Documents/PregnantPrj/Data')
# matched_data.to_csv('matched_data.csv')


w_trauma                                                   \
         Date ID cEmotAbuse Act_Score Avg_dailyHRV Sleep_Score   
0  2023-05-18  5          1        94    73.206897        79.0   
1  2023-05-19  5          1        94    98.931818        77.0   
2  2023-05-20  5          1        92    77.219048        88.0   
3  2023-05-21  5          1        97    75.577586        83.0   
4  2023-05-22  5          1        98    74.925000        77.0   

                     wo_trauma                                        \
  Propensity_Score        Date  ID cEmotAbuse Act_Score Avg_dailyHRV   
0         0.504646  2023-06-01   3          0        82    86.347222   
1         0.516031  2023-06-24  13          0        94    27.372549   
2         0.446885  2023-06-05   3          0        85    70.289157   
3         0.491927  2023-05-18   3          0        92    83.415730   
4         0.529416  2023-05-12   3          0        88    69.180328   

                                
  Sleep_Score Propensity_Score  
0        72.0         0.504270  
1        77.0         0.516031  
2        84.0         0.446199  
3        80.0         0.492245  
4        71.0         0.530050

In [9]:
from sklearn.neighbors import NearestNeighbors

# Separate treated and control groups
PTL_all = df_modify_HRV_all.groupby('Trauma')
control = PTL_all.get_group(0)
treated = PTL_all.get_group(1)

# Find matches for treated using Nearest Neighbors
knn = NearestNeighbors(n_neighbors=1).fit(control[['Propensity_Score']])
distances, indices = knn.kneighbors(treated[['Propensity_Score']])

# Create a dataframe of matched pairs
matched_data = pd.concat([
    treated.reset_index(drop=True),
    control.iloc[indices.flatten()].reset_index(drop=True)
], axis=1, keys=['Treated', 'Control'])

# matched_data.head()

KeyError: 'Trauma'

In [12]:
# Calculate average weight loss in the matched treated and control groups
avg_weight_loss_treated = matched_data['w_trauma']['Avg_dailyHRV'].mean()
avg_weight_loss_control = matched_data['wo_trauma']['Avg_dailyHRV'].mean()

# Estimate average treatment effect (ATE)
ATE = avg_weight_loss_treated - avg_weight_loss_control

avg_weight_loss_treated, avg_weight_loss_control, ATE

(47.14534084272693, 41.79685129134927, 5.348489551377661)

In [18]:
import scipy.stats as stats
# t_statistic, p_value = stats.ttest_1samp(w_trauma['Avg_dailyHRV'], wo_trauma['Avg_dailyHRV'].mean())
t_statistic, p_value = stats.ttest_1samp(matched_data['w_trauma']['Avg_dailyHRV'], matched_data['wo_trauma']['Avg_dailyHRV'].mean() )


t_statistic, p_value

(3.6586450318227888, 0.0003206612284353537)

In [16]:
# Step 4: Analyze
print(matched_data['w_trauma']['Avg_dailyHRV'].mean())
print(matched_data['wo_trauma']['Avg_dailyHRV'].mean())
print(w_trauma['Avg_dailyHRV'].mean())
print(wo_trauma['Avg_dailyHRV'].mean())
# Calculate the average trauma effect among the matched pairs
ate_matched = (matched_data['w_trauma']['Avg_dailyHRV'] - matched_data['wo_trauma']['Avg_dailyHRV']).mean()

# For comparison, calculate the average trauma effect without matching
ate_unmatched = w_trauma['Avg_dailyHRV'].mean() - wo_trauma['Avg_dailyHRV'].mean()

print(ate_matched, ate_unmatched,ate_unmatched-ate_matched)

47.14534084272693
41.79685129134927
47.14534084272693
39.6598296468988
5.348489551377663 7.485511195828131 2.137021644450468


In [102]:
print(df_modify_HRV_all.head())
print("df_act_all: ",len(df_modify_HRV_all))

         Date  ID  Trauma  Act_Score  Avg_dailyHRV  Sleep_Score  \
0  2023-06-10   1       0         75     47.365854         95.0   
1  2023-06-11   1       0         78     80.518519         93.0   
2  2023-06-13   1       0         83     40.669492         95.0   
3  2023-06-17   1       0         83     49.626016         97.0   
4  2023-06-18   1       0         87     12.822222         58.0   

   Propensity_Score  
0          0.541397  
1          0.560487  
2          0.566886  
3          0.557417  
4          0.737401  
df_act_all:  489
